# AEMO Causer Pays Data

Demos using a SplitQuery cache to load and analyse AEMO causer pays data for generators.

In [ ]:
# In lieu of a nice progress bar at this point, log messages show data retrieval.
import logging
logging.basicConfig(level=logging.INFO)

# Stuff we'll need for filtering and plotting.
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime

# Data sets.
from aemo import AEMOCauserPays, AEMOElementNames
# Creates a dynamic dataset that loads data when needed.
dataset = AEMOCauserPays()
# Creates a dataframe (from disk cache if possible).
elements = AEMOElementNames().get()

In [ ]:
# Show element ids for wind farms.
elements[elements.element_name.str.match('SUBSTN.*WF.*GEN')].sort_values('element_name')

# Generation vs Forecast

The AEMO data is stored in files covering 30min intervals.
SplitQuery will load data files required to cover the datetime interval specified in the filter, cache the data for later use, and refine the loaded dataset to match the remaining filters.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
(
    dataset[
        dataset.datetime.between(
            datetime(2018, 8, 1, 21, 0), datetime(2018, 8, 2, 2, 0))
        & dataset.element_id.isin([312, 316, 319])
        & dataset.variable_id.isin([2, 3])]
    .get()
    .set_index(['datetime', 'variable_id', 'element_id'])
    .value
    .unstack().rename(columns=dict(elements.set_index('element_id').element_name))
    .unstack().rename(columns={2: 'Gen_MW', 3: 'AWEFS'})
    .plot(ax=ax)
);

In [ ]:
dataset.clear_cache()